# To run "hierarchicell" package. The version "Matrix" of package must be lower than 1.6.3. If higher than that, need to remove and reinstall. 
## link <- "https://cran.r-project.org/src/contrib/Archive/Matrix/Matrix_1.6-3.tar.gz"
## install.packages(link, repos=NULL, type="source")

In [ ]:
# To run "hierarchicell" package. The version "Matrix" of package must be lower than 1.6.3. If higher than that, need to remove and reinstall. 
link <- "https://cran.r-project.org/src/contrib/Archive/Matrix/Matrix_1.6-3.tar.gz"
link <- "https://cran.r-project.org/src/contrib/Archive/Matrix/Matrix_1.6-4.tar.gz"
install.packages(link, repos=NULL, type="source")

In [ ]:
### Extract the cell barcodes of interest, and generate 40 (customized n) barcodes in a text file to accelerate the speed of step 2
# packages
library(utils)
library(stats)
library(Seurat)
library(Signac)
library(tidyr)
library(dplyr)
library(hierarchicell)
library(stringr)


gc()

In [ ]:
sessionInfo("Matrix")

In [ ]:
## load in metadata for each dataset
sample.meta <- read.csv("./Merge_multi_b2345.csv")
#sample.meta

In [ ]:
## read in the object. The method is simulation based, then will need reference matrix for each cell type in each region. 
obj <- readRDS("./cellbender_EC_object_10.31.rds")
obj

In [ ]:
table(obj$diagnosis,obj$individual_ID)
table(obj$diagnosis,obj$cluster_celltype)

In [ ]:
## get parameters for the simulation and powere analysis
DefaultAssay(obj) <- "PC"
ct <- "Oligodendrocyte"

# get number of cells in each condition
meta <- obj@meta.data
meta_eoad <- meta[meta$diagnosis == "EOAD",]
meta_ctl <- meta[meta$diagnosis == "NCI",]

n_cell_case = ceiling(mean(table(meta_eoad[meta_eoad$cluster_celltype == ct,]$individual_ID)))
n_cell_ctl = ceiling(mean(table(meta_ctl[meta_ctl$cluster_celltype == ct,]$individual_ID)))

# subsetting
obj1 <- subset(obj, subset = cluster_celltype == ct)
obj1

## filter genes
# calculating the number of expressed genes: genes expressed in more than 25% of cells.
counts <- LayerData(obj1,assay = "PC",layer = "counts")# code in seurat 5.0+
genes.percent.expression <- rowMeans(counts>0 )*100   
genes.filter <- names(genes.percent.expression[genes.percent.expression>25])

# number of expressed genes. 
n_genes = length(genes.filter)

###################################
##-- building reference matrix --##
###################################
exp <- as.data.frame(t(counts))

## adding columns
exp$CellID <- rownames(exp)
exp$individualID <- str_split_fixed(rownames(exp),pattern = "_",n = 2)[,1]

## rearrange
exp <- exp[,c(14928:14929,1:14927)]

In [ ]:
## Estimate parameters from the simulated data
clean_exp <- filter_counts(exp)
data_summaries <- compute_data_summaries(clean_exp,type = "Raw")

In [ ]:
approximate_gene_mean(data_summaries,plot= T)
model_dispersion(data_summaries, plot = T)
model_inter(data_summaries, plot = T)
approximate_gene_drop(data_summaries,plot = T)
model_drop_sd(data_summaries,plot = T)

In [ ]:
#print(n_genes)
print(n_cell_case)
print(n_cell_ctl)
print(n_genes)
### Estimate power with a two-part hurdle mixed model
options(warn=-1)
res <- power_hierarchicell(data_summaries,n_genes=3000,cells_per_case = n_cell_case,cells_per_control = n_cell_ctl, n_per_group = 3,foldchange = 1.5,pval = 0.05)
options(warn=0)

# printing results
print(res)

In [ ]:
#print(n_genes)
print(n_cell_case)
print(n_cell_ctl)
print(n_genes)
### Estimate power with a two-part hurdle mixed model
options(warn=-1)
res <- power_hierarchicell(data_summaries,n_genes=3000,cells_per_case = n_cell_case,cells_per_control = n_cell_ctl, n_per_group = 3,foldchange = 2,pval = 0.05)
options(warn=0)

# printing results
print(res)

In [ ]:
clean_expr_data <- filter_counts()
data_summaries <- compute_data_summaries(clean_expr_data)
simulated_counts <- simulate_hierarchicell(data_summaries)

#power_hierarchicell(data_summaries)